## Importing libraries

In [1]:
from nltk.metrics import *
import jellyfish
import mysql.connector 

## Connecting to Database and Cursor creation

In [2]:
db = mysql.connector.connect(user='root',password='',host='localhost',database='test')

cursor = db.cursor(buffered=True)

## Populating the result table

In [3]:
# Checks for matching entries based on URLs in the table

cursor.execute("INSERT INTO book_entity_matches (rf_project_name,rg_project_name)\
                SELECT rf.project_name,rg.project_name\
                FROM test.book_rf_entities rf\
                INNER JOIN test.book_rg_entities rg\
                ON rf.url = rg.url")

IntegrityError: 1062 (23000): Duplicate entry 'aalib-ruby-aalib-ruby' for key 'PRIMARY'

In [4]:
# Checks for matching entries based on matching project names
cursor.execute("INSERT INTO book_entity_matches(rf_project_name, \
                                                rg_project_name) \
                SELECT rf.project_name, rg.project_name \
                FROM test.book_rf_entities rf \
                INNER JOIN test.book_rg_entities rg \
                ON rf.project_name = rg.project_name \
                WHERE rf.project_name NOT IN ( \
                    SELECT bem.rf_project_name \
                    FROM book_entity_matches bem)")

## Utility Queries

#### First query 
returns the dev's username and realname from rf people table WHERE 
either the username or the realname matches the 'person_name' in rg people table

#### Second query
upates the row for (rf project,rg project) names and sets the flags and distances to appropriate values

In [5]:
#first query
peopleQuery = "SELECT rf.dev_username, rf.dev_realname \
               FROM test.book_rf_entity_people rf \
               WHERE rf.project_name =  %s \
               AND (rf.dev_username IN ( \
                    SELECT rg.person_name \
                    FROM test.book_rg_entity_people rg \
                    WHERE rg.project_name =  %s) \
                    OR \
                    rf.dev_realname IN ( \
                    SELECT rg.person_name \
                    FROM test.book_rg_entity_people rg \
                    WHERE rg.project_name = %s))"

In [6]:
#second query
updateQuery = "UPDATE book_entity_matches\
                SET rf_name_soundex = %s,\
                    rg_name_soundex = %s,\
                    url_levenshtein = %s,\
                    name_levenshtein = %s,\
                    rf_name_in_rg_name = %s,\
                    rf_name_in_rg_url = %s,\
                    rf_dev_in_rg_dev = %s\
                WHERE rf_project_name = %s\
                AND rg_project_name = %s"

## Updating the populated table

### Finding the project pairs 

In [7]:
#make the pair and put in projectPairs
cursor.execute("SELECT bem.rf_project_name, bem.rg_project_name, \
                       rfe.url, rge.url \
                FROM test.book_entity_matches bem \
                    INNER JOIN test.book_rg_entities rge \
                      ON bem.rg_project_name = rge.project_name \
                    INNER JOIN test.book_rf_entities rfe \
                      ON bem.rf_project_name = rfe.project_name \
                ORDER BY bem.rf_project_name")

projectPairs = cursor.fetchall()

### Processing the project pairs

In [8]:
# for each pair, fill the other columns as per process.

for(projectPair) in projectPairs:
    RFname = projectPair[0]
    RGname = projectPair[1]
    RFurl = projectPair[2]
    RGurl = projectPair[3]
    
    # lowercase everything
    RFnameLC = RFname.lower()
    RGnameLC = RGname.lower()
    RFurlLC = RFurl.lower()
    RGurlLC = RGurl.lower()
    
    # calculate string metrics
    levNames = edit_distance(RFnameLC,RGnameLC)
    levURLs = edit_distance(RFurlLC,RGurlLC)
    soundexRFname = jellyfish.soundex(RFnameLC)
    soundexRGname = jellyfish.soundex(RGnameLC)
    
    # is the RF project name inside the RG project name?
    if RFnameLC in RGnameLC:
        rf_in_rg = 1
    else:
        rf_in_rg = 0
    
    # is the RF project name inside the RG project URL?
    if RFnameLC in RGurlLC:
        rf_in_rgurl = 1
    else:
        rf_in_rgurl = 0
        
    #is any dev on RF candidate(result table) in the dev list for RG candidate
    cursor.execute(peopleQuery,(RFname,RGname,RGname))
    result = cursor.fetchone()
    if result is not None:
        rfdev_in_rgdev = 1
    else:
        rfdev_in_rgdev = 0
    
    cursor.execute(updateQuery,(soundexRFname,soundexRGname,levURLs,levNames,rf_in_rg,rf_in_rgurl,rfdev_in_rgdev,RFname,RGname))
    

## Calculating Results
### Total number of rows in result table

In [9]:
#Calculate the total rows in result table
cursor.execute("SELECT * FROM book_entity_matches")    
p = cursor.fetchall()
print(f"The total rows in result table is: {len(p)}")

The total rows in result table is: 5400


### True Positives in the result table (Our guess is positive as well as the answer)
Everything matched to requirements

In [10]:
cursor.execute("SELECT rf_project_name,rg_project_name \
                FROM book_entity_matches \
                WHERE name_levenshtein = 0 \
                AND url_levenshtein = 0 \
                AND rf_name_in_rg_url = 1 \
                AND rf_dev_in_rg_dev = 1")

truePositives = cursor.fetchall()
print(f"The number of true positives are {len(truePositives)}")

The number of true positives are 1046


### False Positives in result table (Our guess is positive but result is negative)

##### First Query
Finding the results where names matched but nothing else did
##### Second Query
Finding the results where urls matched but nothing else did

In [11]:
cursor.execute("SELECT rf_project_name, rg_project_name \
                FROM book_entity_matches \
                WHERE name_levenshtein =0 \
                AND url_levenshtein > 0 \
                AND rf_name_in_rg_url =0 \
                AND rf_dev_in_rg_dev =0")
falsePositives1 = cursor.fetchall()
print(f"The number of false positives are {len(falsePositives1)}")

The number of false positives are 106


In [12]:
cursor.execute("SELECT rf_project_name, rg_project_name \
                FROM book_entity_matches \
                WHERE url_levenshtein = 0 \
                AND rf_name_soundex <> rg_name_soundex \
                AND name_levenshtein > 0 \
                AND rf_name_in_rg_name = 0 \
                AND rf_name_in_rg_url = 0 \
                AND rf_dev_in_rg_dev = 0")

falsePositives2 = cursor.fetchall()
print(f"The number of false positives are {len(falsePositives2)}")

The number of false positives are 104


### Finding precision
#### $$Precision = \frac{ tp}{tp+fp}$$

In [13]:
precision = (len(truePositives)/(len(truePositives)+len(falsePositives1)+len(falsePositives2)))
print(f"Precision of our results is: {round(precision,5)*100}%")

Precision of our results is: 83.28%
